In [1]:
import os

# Parameters
input_dir = "./xyz_files"
output_dir = "./scan"
delta = 0.05
scales = [1 + i * delta for i in range(-5, 30)]  # e.g., from 0.75 to 2.45

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if not file.endswith(".xyz"):
        continue

    filepath = os.path.join(input_dir, file)
    base_name = os.path.splitext(file)[0]
    output_xyz_path = os.path.join(output_dir, base_name + ".xyz")

    # Read original .xyz file
    with open(filepath) as f:
        lines = f.readlines()

    natoms = int(lines[0])
    coords = []
    symbols = []

    for line in lines[2:2+natoms]:
        s, x, y, z = line.split()
        symbols.append(s)
        coords.append([float(x), float(y), float(z)])

    # Write scaled geometries to one .xyz file
    with open(output_xyz_path, "w") as out:
        for i, scale in enumerate(scales):
            out.write(f"{natoms}\n")
            out.write(f"scale = {scale:.5f}\n")
            for atom, (x, y, z) in zip(symbols, coords):
                new_x, new_y, new_z = x * scale, y * scale, z * scale
                out.write(f"{atom:2s}  {new_x:.10f}  {new_y:.10f}  {new_z:.10f}\n")

    print(f"✅ Written scan file: {output_xyz_path}")

print("🎉 All XYZ scan files generated successfully.")


✅ Written scan file: ./scan/LiTb3_1185471.xyz
✅ Written scan file: ./scan/Li3Ce_1185231.xyz
✅ Written scan file: ./scan/LiMg2_1094604.xyz
✅ Written scan file: ./scan/Li3N2_1222421.xyz
✅ Written scan file: ./scan/Li3Rh_1185235.xyz
✅ Written scan file: ./scan/Li2Ga7_1222769.xyz
✅ Written scan file: ./scan/LiPm3_1185395.xyz
✅ Written scan file: ./scan/K3Li_1184918.xyz
✅ Written scan file: ./scan/LiBe_2475638.xyz
✅ Written scan file: ./scan/LiHg_2012.xyz
✅ Written scan file: ./scan/Li21Si5_29720.xyz
✅ Written scan file: ./scan/Li3La_1185192.xyz
✅ Written scan file: ./scan/LiMg_976239.xyz
✅ Written scan file: ./scan/LiY3_1185240.xyz
✅ Written scan file: ./scan/LiN_1059612.xyz
✅ Written scan file: ./scan/K3Li_1184845.xyz
✅ Written scan file: ./scan/Li7Al6_1212183.xyz
✅ Written scan file: ./scan/Li8Hf3_1190933.xyz
✅ Written scan file: ./scan/LiGe3_1185336.xyz
✅ Written scan file: ./scan/LiMg2_1094605.xyz
✅ Written scan file: ./scan/LiPt2_30764.xyz
✅ Written scan file: ./scan/Rb3Li_974938.xyz


In [2]:
import numpy as np
import csv

def extract_scan_data_to_csv(input_xyz, d3s_file, d3_file, output_csv):
    """
    Parse XYZ trajectory and D3/D3S energies to produce a CSV with frame cell lengths and energies.

    Parameters:
    - input_xyz (str): Path to the .xyz scan file.
    - d3s_file (str): Path to the D3S energy file.
    - d3_file (str): Path to the D3 energy file.
    - output_csv (str): Output CSV file path.
    """
    # Read D3S values
    with open(d3s_file, "r") as f:
        d3s_values = [float(line.strip()) for line in f if line.strip()]

    # Read D3 values
    with open(d3_file, "r") as f:
        d3_values = [float(line.strip()) for line in f if line.strip()]

    # Prepare XYZ parsing
    with open(input_xyz, "r") as f:
        lines = f.readlines()

    i = 0
    frame_idx = 0
    cell_lengths = []

    while i < len(lines):
        try:
            natoms = int(lines[i].strip())
            comment = lines[i + 1].strip()
            frame_lines = lines[i + 2:i + 2 + natoms]

            coords = []
            for line in frame_lines:
                parts = line.split()
                coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
            
            coords = np.array(coords)

            # Get extents in x, y, z
            x_length = np.max(coords[:, 0]) - np.min(coords[:, 0])
            y_length = np.max(coords[:, 1]) - np.min(coords[:, 1])
            z_length = np.max(coords[:, 2]) - np.min(coords[:, 2])

            scale = comment.split("scale = ")[1] if "scale =" in comment else "?"

            # Get corresponding D3 value (if available)
            d3s_value = d3s_values[frame_idx] if frame_idx < len(d3s_values) else "NA"
            d3_value = d3_values[frame_idx] if frame_idx < len(d3_values) else "NA"

            # Collect row
            cell_lengths.append([frame_idx, scale, x_length, y_length, z_length, d3s_value, d3_value])

            # Next frame
            i += natoms + 2
            frame_idx += 1

        except Exception as e:
            print(f"❌ Error at frame {frame_idx}: {e}")
            break

    # Write to CSV
    with open(output_csv, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Frame", "Scale", "a_length (Å)", "b_length (Å)", "c_length (Å)", "D3s_Energy", "D3_Energy"])
        writer.writerows(cell_lengths)

    print(f"✅ CSV with D3 energy written to {output_csv}")


In [3]:
import os
import glob

# Save current directory
curr_dir = os.getcwd()

scan_dir = os.path.join(curr_dir, "scan")

# make a folder named output if it doesn't exist
output_dir = os.path.join(curr_dir, "output")
os.makedirs(output_dir, exist_ok=True)

# Find all .xyz files in scan/
xyz_files = glob.glob(os.path.join(scan_dir, "*.xyz"))

print(scan_dir)
print(f"Found {len(xyz_files)} scan files in {scan_dir}")
# Go up one level where Test_script.py is located
os.chdir("..")

# Run Test_script.py for each scan.xyz
for xyz_file in xyz_files:
    print(f"Running Test_script.py on {xyz_file}")

    os.system(f"python Test_script.py \"{xyz_file}\"")

    output_csv_file = f"calcs/output/{os.path.basename(xyz_file).replace('.xyz', '.csv')}"
    extract_scan_data_to_csv(
        input_xyz=xyz_file,
        d3s_file="D3S_BJ_scan.txt",
        d3_file="D3_BJ_scan.txt",
        output_csv=output_csv_file
    )

# Return to original directory
os.chdir(curr_dir)

print("✅ All scans processed.")


/Users/partha/Desktop/research/D3S/calcs/scan
Found 452 scan files in /Users/partha/Desktop/research/D3S/calcs/scan
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiTb3_1185471.xyz
✅ CSV with D3 energy written to calcs/output/LiTb3_1185471.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li3Ce_1185231.xyz
✅ CSV with D3 energy written to calcs/output/Li3Ce_1185231.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiMg2_1094604.xyz
✅ CSV with D3 energy written to calcs/output/LiMg2_1094604.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li3N2_1222421.xyz
✅ CSV with D3 energy written to calcs/output/Li3N2_1222421.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li3Rh_1185235.xyz
✅ CSV with D3 energy written to calcs/output/Li3Rh_1185235.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li2Ga7_1222769.xyz
✅ CSV with D3 energy written

/Users/partha/Desktop/research/D3S/D3S/D3S.py:126: RuntimeWarning: invalid value encountered in scalar divide
  return Z(I,J,CN_I,CN_J,k3)/W(I,J,CN_I,CN_J,k3)


✅ CSV with D3 energy written to calcs/output/Li21Si5_29720.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li3La_1185192.xyz
✅ CSV with D3 energy written to calcs/output/Li3La_1185192.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiMg_976239.xyz
✅ CSV with D3 energy written to calcs/output/LiMg_976239.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiY3_1185240.xyz
✅ CSV with D3 energy written to calcs/output/LiY3_1185240.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiN_1059612.xyz
✅ CSV with D3 energy written to calcs/output/LiN_1059612.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/K3Li_1184845.xyz
✅ CSV with D3 energy written to calcs/output/K3Li_1184845.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li7Al6_1212183.xyz
✅ CSV with D3 energy written to calcs/output/Li7Al6_1212183.csv
Running Test_script.py on /Us

/Users/partha/Desktop/research/D3S/D3S/D3S.py:126: RuntimeWarning: invalid value encountered in scalar divide
  return Z(I,J,CN_I,CN_J,k3)/W(I,J,CN_I,CN_J,k3)


✅ CSV with D3 energy written to calcs/output/Li7Si3_1201871.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiMg5_1094609.xyz
✅ CSV with D3 energy written to calcs/output/LiMg5_1094609.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/LiTl_1191883.xyz


...

Traceback (most recent call last):
  File "/Users/partha/Desktop/research/D3S/Test_script.py", line 13, in <module>
    S6,S8,ALPHA1,ALPHA2 = D3S.DFT_D3_parameters["BLYP"]
                          ^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'D3S.D3S' has no attribute 'DFT_D3_parameters'. Did you mean: 'DFT_D3BJ_parameters'?


✅ CSV with D3 energy written to calcs/output/Li2Mg_1094601.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li5Mg_977122.xyz


Traceback (most recent call last):
  File "/Users/partha/Desktop/research/D3S/Test_script.py", line 13, in <module>
    S6,S8,ALPHA1,ALPHA2 = D3S.DFT_D3_parameters["BLYP"]
                          ^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'D3S.D3S' has no attribute 'DFT_D3_parameters'. Did you mean: 'DFT_D3BJ_parameters'?


✅ CSV with D3 energy written to calcs/output/Li5Mg_977122.csv
Running Test_script.py on /Users/partha/Desktop/research/D3S/calcs/scan/Li3Lu_1185208.xyz
✅ CSV with D3 energy written to calcs/output/Li3Lu_1185208.csv
✅ All scans processed.


Traceback (most recent call last):
  File "/Users/partha/Desktop/research/D3S/Test_script.py", line 13, in <module>
    S6,S8,ALPHA1,ALPHA2 = D3S.DFT_D3_parameters["BLYP"]
                          ^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'D3S.D3S' has no attribute 'DFT_D3_parameters'. Did you mean: 'DFT_D3BJ_parameters'?


In [ ]:
import numpy as np
import csv

input_xyz = "scan.xyz"
output_csv = "cell_lengths.csv"
d3s_file = "../D3S_BJ_scan.txt"
d3_file = "../D3_BJ_scan.txt"

# Read D3S values
with open(d3s_file, "r") as f:
    d3s_values = [float(line.strip()) for line in f if line.strip()]

# Read D3 values
with open(d3_file, "r") as f:
    d3_values = [float(line.strip()) for line in f if line.strip()]

# Prepare XYZ parsing
with open(input_xyz, "r") as f:
    lines = f.readlines()

i = 0
frame_idx = 0
cell_lengths = []

while i < len(lines):
    try:
        natoms = int(lines[i].strip())
        comment = lines[i + 1].strip()
        frame_lines = lines[i + 2:i + 2 + natoms]

        coords = []
        for line in frame_lines:
            parts = line.split()
            coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
        
        coords = np.array(coords)
        
        # Get extents in x, y, z
        x_length = np.max(coords[:, 0]) - np.min(coords[:, 0])
        y_length = np.max(coords[:, 1]) - np.min(coords[:, 1])
        z_length = np.max(coords[:, 2]) - np.min(coords[:, 2])

        scale = comment.split("scale = ")[1] if "scale =" in comment else "?"

        # Get corresponding D3 value (if available)
        d3s_value = d3s_values[frame_idx] if frame_idx < len(d3s_values) else "NA"
        d3_value = d3_values[frame_idx] if frame_idx < len(d3_values) else "NA"
        # Collect row
        cell_lengths.append([frame_idx, scale, x_length, y_length, z_length, d3s_value,  d3_value])

        # Next frame
        i += natoms + 2
        frame_idx += 1

    except Exception as e:
        print(f"Error at frame {frame_idx}: {e}")
        break

# Write to CSV
with open(output_csv, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Frame", "Scale", "a_length (Å)", "b_length (Å)", "c_length (Å)", "D3s_Energy,D3_energy"])
    writer.writerows(cell_lengths)

print(f"✅ CSV with D3 energy written to {output_csv}")


✅ CSV with D3 energy written to cell_lengths.csv
